1. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát éves bontásban norbert2 azonosítójú ügyfél rendeléseinek értékéről!</span>

1. A lista megfelelően jelölve tartalmazza a végösszeget is!

In [4]:
-- rendelés és rendelés_tétel táblák
-- érték: SUM(mennyiseg*egysegar)
SELECT IIF(YEAR(r.REND_DATUM) is null, 'Összesen',Cast(Year(r.REND_DATUM) as nvarchar(4))),
    SUM(rt.MENNYISEG)
    from Rendeles r join Rendeles_tetel rt on r.SORSZAM = rt.SORSZAM
    where r.[LOGIN]='norbert2'
    group by rollup(year(r.REND_DATUM))

(3 rows affected)

Total execution time: 00:00:00.030

(No column name),(No column name)
2015,871
2017,1150
Összesen,2021


2. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát szállítási dátumonként, azon belül szállítási módonként az egyes rendelések összmennyiségéről!</span>

1. Csak azokat a termékeket vegyük figyelembe, amelyek mennyiségi egysége db!
2. A listát szűrjük úgy, hogy az csak a részösszeg sorokat és a végösszeget tartalmazza!

In [ ]:
-- rendeles, rendeles_tetel, termék táblák
-- SUM(mennyiseg)
SELECT r.SZALL_DATUM,r.SZALL_MOD, SUM(rt.MENNYISEG)
    from    Rendeles r  join Rendeles_tetel rt on r.SORSZAM = rt.SORSZAM
                        join Termek t on rt.TERMEKKOD = t.TERMEKKOD
    where t.MEGYS ='db'
    group by rollup(r.SZALL_DATUM,r.SZALL_MOD)
    HAVING r.SZALL_MOD is null or r.SZALL_DATUM is NULL

```
3. Hány olyan ügyfél van, aki még nem rendelt semmit?
```

1. Csoportosítsuk őket nem szerint, azon belül életkor szerint!
2. A lista tartalmazza a részösszegeket és a végösszeget is!

![](https://moodle.uni-corvinus.hu/theme/image.php/adaptable/atto_h5p/1709548862/icon)

In [21]:
SELECT u.nem,
       YEAR(GETDATE()) - u.SZULEV AS 'életkor',
       COUNT(*)
FROM Ugyfel u LEFT join Rendeles r on u.[LOGIN]=r.[LOGIN]
WHERE r.[LOGIN] is null
GROUP BY ROLLUP(u.nem, YEAR(GETDATE())- u.SZULEV)


(12 rows affected)

Total execution time: 00:00:00.037

nem,életkor,(No column name)
F,24,1
F,26,1
F,28,1
F,45,1
F,55,1
F,56,1
F,NULL,6
N,30,1
N,37,1
N,49,1


4. <span style="color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a megrendelt termékek legkisebb és legnagyobb egységáráról szállítási dátum, azon belül szállítási mód szerinti bontásban!</span>

1. A lista csak a 2015 májusi szállításokat tartalmazza!
2. Jelenítsük meg a részösszegeket és a végösszeget is!

In [ ]:
SELECT  MIN(rt.EGYSEGAR) as "Legkisebb egységár",
        MAX(rt.EGYSEGAR) as "Legnagyobb egységár",
        r.SZALL_DATUM,
        r.SZALL_MOD
from Rendeles r join Rendeles_tetel rt on r.SORSZAM = rt.SORSZAM
where YEAR(r.SZALL_DATUM)=2015 and MONTH(r.SZALL_DATUM)=05
group by rollup(r.SZALL_DATUM,r.SZALL_MOD)

5. <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk csoportot a termékek listaára alapján a következők szerint:&nbsp;<br></span> <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Az "olcsó" termékek legyenek azok, amelyek listaára 3000 alatt van.<br>&nbsp;A "drága" termékek legyenek az 5000 felettiek, a többi legyen "közepes".</span>

1. Listázzuk az egyes csoportokat, és a csoportokba tartozó termékek darabszámát!
2. A lista jelenítse meg a végösszeget is!

In [30]:
SELECT CASE
        When LISTAAR > 5000 then 'Drága'
        When LISTAAR < 3000 then 'Olcsó'
        Else 'Közepes'
        END as 'Csoport',
        COUNT(*) as 'DB'
FROM Termek
group by rollup(Case
        When LISTAAR > 5000 then 'Drága'
        When LISTAAR < 3000 then 'Olcsó'
        Else 'Közepes'
        END)

(4 rows affected)

Total execution time: 00:00:00.021

Csoport,DB
Drága,20
Közepes,5
Olcsó,526
NULL,551


6. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát az ügyfelek adatairól név szerinti sorrendben.</span>

1. Minden sorban jelenjen meg a sorrend szerint előző, illetve következő ügyfél neve is.
2. Ha nincs előző vagy következő ügyfél, akkor a 'Nincs' jelenjen meg!

In [ ]:
--lag és lead fv-ek
SELECT  *,
        LAG(u.nev,1, 'Nincs') over(order by nev) as 'Előző ügyfél',
        Lead(u.nev,1, 'Nincs') over(order by nev) as 'Következő ügyfél'
from Ugyfel u

<mark>7. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk lekérdezést, amely megmutatja, hogy melyik termékkategóriába hány termék tartozik. A lista a kategória nevét és a darabszámot jelenítse meg.</span></mark>
1. A lista ne tartalmazzon duplikált sorokat.
2. A feladatot partíciók segítségével oldjuk meg!

In [ ]:
select DISTINCT tk.kat_nev,
        COUNT(t.kat_id) over (PARTITION by t.kat_id)
from Termek t join Termekkategoria tk on t.KAT_ID = tk.KAT_ID


8. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről. Az egyes rendelési tételeket termékenként soroljuk be 4 osztályba a rendelés mennyisége alapján. Jelenítsük meg ezt az információt is egy új oszlopban, az oszlop neve legyen 'Mennyiségi kategória'.</span>

1. A lista csak a 100 Ft feletti egységárú rendelési tételeket vegye figyelembe!

In [ ]:
--ntile()
select  *,
        NTILE(4) over(PARTITION by termekkod order by mennyiseg)
from Rendeles_tetel
where EGYSEGAR > 100

9. <span style="font-weight: 600; color: rgb(36, 41, 47); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, megnevezését, kategóriájának nevét, és listaárát.</span>

1. A listát egészítsük ki két új oszloppal, amelyek a kategória legolcsóbb, illetve legdrágább termékének árát tartalmazzák.
2. A két új oszlop létrehozásánál partíciókkal dolgozzunk!

In [ ]:
--min/max vagy first_value/last_value
select t.termekkod,
        t.megnevezes,
        tk.kat_nev,
        t.listaar,
        min(t.listaar) over(partition by t.KAT_ID),
        max(t.listaar) over(partition by t.KAT_ID)
from termek t join termekkategoria tk on t.kat_id = tk.KAT_ID

10. <span style="font-weight: 600; color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelésekről. A lista legyen rendezve ügyfelenként (LOGIN), azon belül a rendelés dátuma szerint. A listához készítsünk sorszámozást is. A sorszám a következő formában jelenjen meg: sorszám_év_login. Pl: 1_2015_adam1</span>

1. A számozás login-onként, azon belül rendelési évenként kezdődjön újra.
2. A sorszám oszlop neve legyen Azonosító

In [ ]:
select CAST(ROW_NUMBER() over(partition by r.LOGIN, YEAR(R.REND_DATUM) order by r.REND_DATUM) as nvarchar(5))+'_'+
        CAST(R.REND_DATUM as nvarchar(4)) + '_' + r.[LOGIN] as 'Azonosító',
        *
from Rendeles r


11. <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek adatairól listaár szerint növekvő sorrendben! A lista jelenítse meg két új oszlopban a sorrend szerint előző, illetve következő termék listaárát is a termék saját kategóriájában és raktárában!</span>

1. Ahol nincs előző vagy következő érték, ott 0 jelenjen meg!
2. Az oszlopokat nevezzük el értelemszerűen!

In [18]:
select  *,
        lag(listaar,1,0) over (partition by kat_id, RAKTAR_KOD order by listaar ) as 'Előző',
        lead(listaar,1,0) over (partition by kat_id, RAKTAR_KOD order by listaar) as 'Következő'
from Termek
order by LISTAAR

12. <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Listázzuk a termékek kódját, nevét és listaárát listaár szerinti sorrendben!</span>

1. Vegyünk fel egy új oszlopot Mozgóátlag néven, amely minden esetben az aktuális termék, az előző, és a következő termék átlagárát tartalmazza!
2. A mozgóátlagot kerekítsük két tizedesre!

In [ ]:
select termekkod,
        megnevezes,
        listaar,
        --mozgóátlag
        avg(Listaar) over(order by listaar
                rows between 1 preceding and 1 following)
from termek

13. <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát, amely a rendelések sorszámát és a rendelés értékét tartalmazza. A listát egészítsük ki egy új oszloppal, amely minden rendelés esetén addigi rendelések összegét tartalmazza (az aktuálisat is beleértve)!</span>

1. A listát rendezzük sorszám szerint növekvő sorrendbe.
2. A lista ne tartalmazzon duplikált sorokat!

In [36]:
select  DISTINCT sorszam,
        SUM(MENNYISEG*EGYSEGAR) over(partition by sorszam) as 'rendelés érték',
        SUM(MENNYISEG*EGYSEGAR) over(order by sorszam range between unbounded PRECEDING and CURRENT ROW)
from Rendeles_tetel
order by SORSZAM

14. <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a termékek kódjáról, nevéről, kategória azonosítójáról, raktár azonosítójáról és listaáráról, valamint a termék adott szempontok szerinti rangsorokban elfoglalt helyezéseiről. (Szempontonként külön oszlopban, a helyezéseknél növekvő sorrendet feltételezve). A szempontok a következők legyenek: listaár, kategória szerinti listaár, és raktárkód szerinti listaár.</span>

1. Az oszlopokat nevezzük el értelemszerűen.
2. A helyezések egyenlőség esetén "sűrűn" kövessék egymást.
3. A lista legyen rendezett kategória azonosító, azon belül listaár szerint!

15. <span style="color: rgb(255, 255, 255); font-family: -apple-system, BlinkMacSystemFont, &quot;Segoe UI&quot;, Helvetica, Arial, sans-serif, &quot;Apple Color Emoji&quot;, &quot;Segoe UI Emoji&quot;; font-size: 16px;">Készítsünk listát a rendelési tételekről, amely minden sor esetén göngyölítve tartalmazza az ügyfél adott rendelési tételig meglévő rendelési tételeinek összértékét!</span>

1. Az új oszlop neve legyen Eddigi rendelési tételek összértéke!
2. Az ügyfél neve is jelenjen meg!